### **Потрошитель этих ваших OIDов**   
* файлы справочников скачать и as is положить в `любую папку`. **Файлы только извлечь из архива и не переименовывать, в справочниках ничего не изменять**;   
* во избежание недоразумений `любую папку` рекомендуется размещать где-то недалеко от корня диска;   
* в ту же папку положить список oid для расшепления. **Файл назвать oid.xlsx, в нём должен быть единственный рабочий лист по имени oid**.

In [ ]:
%%time

import pandas as pd
import os

# игнорирование предупреждения о формате при импорте из excel
import warnings

warnings.simplefilter("ignore")


def real_name(row):
    """вспомогательная функция для формирования 'правильного названия кабинета'"""
    try:
        res = row[
            [
                "hospital_subdivision_name",
                "ambulance_subdivision_name",
                "lab_subdivision_name",
            ]
        ]
        return pd.DataFrame(res).dropna().iloc[0].tolist()[0]
    except Exception:
        return "error"


# пути к файлам
dir_path = os.getcwd()
dir_ = os.listdir(dir_path.replace("\\", "/"))
for file in dir_:
    if file.split("_")[0].split(".")[-1] == "1461":
        path_mo = dir_path.replace("\\", "/") + "/" + file
    if file.split("_")[0].split(".")[-1] == "114":
        path_sp = dir_path.replace("\\", "/") + "/" + file
    if file.split("_")[0].split(".")[-1] == "115":
        path_room = dir_path.replace("\\", "/") + "/" + file
path_oid = dir_path.replace("\\", "/") + "/oid.xlsx"
path_save = dir_path.replace("\\", "/") + "/result.csv"
path_save_error = dir_path.replace("\\", "/") + "/error_oid.csv"

print("Загрузка и подготовка справочников...")

# выбор полей справочников для загрузки
col_mo = [
    "oid",
    "nameShort",
    "regionName",
    "moAgencyKind",
    "profileAgencyKind",
    "moDeptName",
]
col_sp = ["mo_oid", "depart_oid", "depart_name", "depart_type_name", "depart_kind_name"]
col_room = [
    "mo_oid",
    "depart_oid",
    "oid",
    "hospital_subdivision_name",
    "ambulance_subdivision_name",
    "lab_subdivision_name",
    "ambulance_room_count",
    "hospital_name",
]

# загрузка и подготовка справочников
df_oid = pd.read_excel(
    path_oid, sheet_name="oid", header=None, index_col=None, dtype="str", names=["oid"]
)
df_mo = pd.read_excel(
    path_mo, usecols=col_mo, sheet_name="Справочник", index_col=None, dtype="str"
)
df_sp = pd.read_excel(
    path_sp, usecols=col_sp, sheet_name="Справочник", index_col=None, dtype="str"
)
df_room = pd.read_excel(
    path_room, usecols=col_room, sheet_name="Справочник", index_col=None, dtype="str"
)
df_mo.rename(columns={"oid": "mo_oid"})
df_room["real_name"] = df_room.apply(real_name, axis=1)
df_room = df_room.rename(columns={"oid": "room_oid"})
df_mo = df_mo.rename(columns={"oid": "mo_oid"})
df_room["ambulance_room_count"] = df_room["ambulance_room_count"].fillna(0)

print("Справочники загружены и сформированы")

In [5]:
%%time

# соединение списка oid и данных МО
df_result_mo = df_oid.merge(df_mo, left_on="oid", right_on="mo_oid", how="left")
# проверим, есть ли нераспознанные oidы и сохрани то, что получилось, в df_oid_error
df_result_mo_test = df_result_mo[["oid", "mo_oid"]]
df_oid_error = df_result_mo_test[df_result_mo_test.isna().any(axis=1)]["oid"]

# вывод нераспознанных oid на экран и запись в файл
if df_oid_error.empty:
    print("Все oid распознаны")
else:
    print("Не распознаны oid:")
    print(df_oid_error.to_string(index=False))
    df_oid_error.to_csv(path_save_error, sep=";", encoding="utf-8-sig", index=False)
    print("Не распознанные oid сохранены в файл error_oid.csv")

# удаление из результата строк с нераспознанными oid
df_result_mo = df_result_mo.loc[~df_result_mo["mo_oid"].isna()]

# соединение таблиц и обработка результатов
df_room = df_room[
    [
        "mo_oid",
        "depart_oid",
        "room_oid",
        "hospital_name",
        "ambulance_room_count",
        "real_name",
    ]
]
df_result_0 = df_result_mo.merge(df_sp, left_on="mo_oid", right_on="mo_oid", how="left")
df_result = df_result_0.merge(
    df_room, left_on="depart_oid", right_on="depart_oid", how="left"
)
df_result = df_result.drop_duplicates().fillna("").reset_index(drop=True)
df_result = df_result[
    [
        "oid",
        "mo_oid_x",
        "depart_oid",
        "room_oid",
        "nameShort",
        "regionName",
        "moDeptName",
        "moAgencyKind",
        "profileAgencyKind",
        "depart_name",
        "depart_type_name",
        "depart_kind_name",
        "hospital_name",
        "ambulance_room_count",
        "real_name",
    ]
]
df_result = df_result.rename(columns={"oid": "source_oid", "mo_oid_x": "mo_oid"})
df_result = df_result.drop_duplicates()

# сохранение
df_result.to_csv(path_save, sep=";", encoding="utf-8-sig", index=False)
print("Результаты расщепления сохранены в файл result.csv")

Не распознаны oid:
1.2.643.5.1.13.13.12.2.22.1609.0.61274.165807
1.2.643.5.1.13.13.12.2.22.1609.0.61274.165808
1.2.643.5.1.13.13.12.2.22.1609.0.61274.165809
1.2.643.5.1.13.13.12.2.22.1609.0.61274.165810
1.2.643.5.1.13.13.12.2.22.1609.0.61274.165811
1.2.643.5.1.13.13.12.2.22.1609.0.61274.165812
1.2.643.5.1.13.13.12.2.22.1609.0.61274.165813
Не распознанные oid сохранены в файл error_oid.csv
Результаты расщепления сохранены в файл result.csv
CPU times: total: 31.2 ms
Wall time: 76.9 ms
